In [ ]:
#pip install torch
#pip install torchvision

In [1]:
#https://github.com/nwams/Hidden_Markov_Model-in-PyTorch/blob/master/Hidden%20Markov%20Models%20in%20PyTorch.ipynb

In [2]:
#Hidden markov model using PyTorch library (deep learning neural networks).
#Graphical HMM models included in this example: Viterbi, Forward-Backward ו-Baum Welch

In [1]:
import pandas as pd

In [2]:
#טבלת תכנות דינמית  נועדה להפוך את האלגוריתם ליעיל יותר על ידי אחסון חלק מתוצאות הביניים. זה שימושי כאשר לאלגוריתם יש הרבה חישובים שחוזרים על עצמם
def dptable(state_prob):
    print(" ".join(("%8d" % i) for i in range(state_prob.shape[0])))
    for i, prob in enumerate(state_prob.T):
        print("%.7s: " % states[i] +" ".join("%.7s" % ("%f" % p) for p in prob))

In [3]:
#Model Parameters
#Transition probability:

#A matrix with the probabilities from transitioning from one state to the next state, over time.
#P(nextState|currentState)
#Emission probability:

#A matrix with the probabilities of an observation (output) being generated from a state.
#P(Observation|currentState)
#Initial probability:

#A starting probability distribution over states
#P(initialState)
#Final probability: A final probability distribution over states

In [4]:
#הוא אלגוריתם דינמי המאפשר לנו לחשב את הנתיב הסביר ביותר. זה לוקח את המקסימום על פני הסתברויות הנתיב הקודם  - Viterbi 
#חבילה זו תחשב באופן רקורסיבי את ההסתברות לנתיב הסביר ביותר
#נתון:
#שני מצבים אפשריים: בריא ו חום
#תצפיות אפשריות: טמפרטורה רגילה, טמפרטורה קרה או סחרחורת
#המודל נעשה שימוש לחיזוי האם האדם חולה או בריא
#ישנם מספר נתיבים דרך המצבים הנסתרים (בריא וחום) המובילים לרצף הנתון, לכל אחד נתיבים שונים

import numpy as np
# A start distribution π is required; setting π = [0.6, 0.4] means a probability 0.6 of starting in state 1 (Healthy), and a  probability 0.4 of starting in state 2 (Fever)
p0 = np.array([0.6, 0.4])

transition = np.array([[0.7, 0.3],
                       [0.4, 0.6]])

emissions = np.array([[0.5, 0.1], 
              [0.4, 0.3], 
              [0.1, 0.6]])

states = {0:'Healthy', 1:'Fever'}
observations = {0:'normal', 1:'cold', 2:'dizzy'}

obs_seq = np.array([0, 0, 1, 2, 2]) # Sequence: Normal, Normal, Cold, Dizzy, Dizzy

In [5]:
#Initial state vector
#We can think of the i_th component of the vector as representing the probability that the chain starts in state $s{i}$

In [6]:
df_p0 = pd.DataFrame(p0, index=["Healthy", "Fever"], columns=["Probability"])
df_p0

,Probability
Healthy,0.6
Fever,0.4


In [7]:
#Transition probability matrix
#Let's assume that if the person is Healthy now the probability that they will be healthy the next "time" is 70%. So that leaves a 30% chance that the person will be Feverish the next "time".

#Let's assume that if the person is Feverish now, there's a 40% chance that they'll become Healthy next "time" and a 60% chance that they'll remain feverish next "time".

In [8]:
df_transition = pd.DataFrame(transition, index=["fromHealthy", "fromFever"], columns=["toHealthy", "toFever"])
df_transition

,toHealthy,toFever
fromHealthy,0.7,0.3
fromFever,0.4,0.6


In [9]:
#Emission probability marix (Observation)
#Let's say that we have concluded that when a person is Healthy there's a 50% chance that they are Normal, a 40% chance that they are Cold, and a 10% chance that they are Dizzy.

#Let's say that we also concluded that when a person is Feverish there's a small 10% chance they are Normal, a 30% chance that they are Cold, and a much larger 60% chance that they are Dizzy.

In [10]:
df_emissions = pd.DataFrame(emissions, index=["Normal", "Cold", "Dizzy"], columns=["Healthy", "Fever"])
df_emissions

,Healthy,Fever
Normal,0.5,0.1
Cold,0.4,0.3
Dizzy,0.1,0.6


In [11]:
#Run Viterbi
#Calculate the Viterbi path, which is the most likely sequence of states that generated the sequence given the full model

In [12]:
from HiddenMarkovModel import HiddenMarkovModel

In [13]:
model = HiddenMarkovModel(transition, emissions, p0) # Define the HMM

states_seq, state_prob = model.viterbi_inference(obs_seq) # Calculate the Viterbi path

In [14]:
print("Observation sequences: ", [observations[o] for o in obs_seq])


Observation sequences:  ['normal', 'normal', 'cold', 'dizzy', 'dizzy']


In [15]:
import torch

# Transpose the state probability matrix. Move the tensor to cpu(). Convert torch tensor to numpy array. 
df = pd.DataFrame(torch.t(state_prob).cpu().numpy(), index=["Health", "Fever"])

def highlight_max(s):
    '''
    Highlight the maximum in a series in yellow.
    '''
    is_max = s == s.max()
    return ['background-color: yellow' if v else '' for v in is_max]

# Display the tensor
df.style.apply(highlight_max, axis=0)

,0,1,2,3,4
Health,0.300000,0.105000,0.029400,0.002058,0.000212
Fever,0.040000,0.009000,0.009450,0.005292,0.001905


In [16]:
print("The most likely states are: ", [states[s.item()] for s in states_seq])


The most likely states are:  ['Healthy', 'Healthy', 'Healthy', 'Fever', 'Fever']


In [17]:
####################################### OTHER EXAMPLE Forward-Backward Example
# Let's consider a situation where you work in a shopping mall with no views to the outside. And you want to infer the weather outside (Rain, No Rain) at the present moment, given only observations of passerby with or without an umbrella.

In [18]:
#Forward-Backward Algorithm (Unsupervised)
#Unlike the part-of-speech tagger (supervised) that has a corpus of labeled words, many applications don’t have labeled data so instead we can use the Forward-Backward algorithm (an unsupervised algorithm).

#The real problem is we don’t know the counts of being in any of the hidden states! The Baum-Welch algorithm solves this by iteratively estimating the counts. We will start with an estimate for the transition and observation probabilities and then use these estimated probabilities to derive better and better probabilities. And we’re going to do this by computing the forward probability for an observation and then dividing that probability mass among all the different paths that contributed to this forward probability. [Pg. 11, Jurafsky]

#Overview

#The goal of the forward-backward algorithm is to find the conditional distribution over hidden states given the data.
#It is used to find the most likely state for any point in time.
#It cannot, however, be used to find the most likely sequence of states (see Viterbi)
#The algorithm makes use of the principle of dynamic programming to compute efficiently the values that are required to obtain the posterior marginal distributions in two passes.
#The first pass goes forward in time while the second goes backward in time.
#[Youtube explanation] of the algorithm.

#Forward-Backward Example
#Let's consider a situation where you work in a shopping mall with no views to the outside. And you want to infer the weather outside (Rain, No Rain) at the present moment, given only observations of passerby with or without an umbrella.

In [19]:
#Define Model Parameters

In [20]:
#Define Model Parameters
p0 = np.array([0.5, 0.5])

emi = np.array([[0.9, 0.2],
                [0.1, 0.8]])

trans = np.array([[0.7, 0.3],
                  [0.3, 0.7]])

states = {0:'rain', 1:'no_rain'}
obs = {0:'umbrella', 1:'no_umbrella'}

obs_seq = np.array([1, 1, 0, 0, 0, 1])

In [21]:
print("Observation sequence: ", [obs[o] for o in obs_seq])


Observation sequence:  ['no_umbrella', 'no_umbrella', 'umbrella', 'umbrella', 'umbrella', 'no_umbrella']


In [22]:
#Run Forward-Backward

In [23]:
model =  HiddenMarkovModel(trans, emi, p0)

In [24]:
model.N = len(obs_seq)

# As defined in the module, Model.S = Number of possible states
shape = [model.N, model.S] # [6, 2]

In [25]:
model.initialize_forw_back_variables(shape) # Creates the "forward", "backward" and "posterior" tensor's each filled with 0's, each with shape [6,2] 


In [26]:
obs_prob_seq = model.E[obs_seq] # Emission probability matrix
obs_prob_seq

tensor([[0.1000, 0.8000],
        [0.1000, 0.8000],
        [0.9000, 0.2000],
        [0.9000, 0.2000],
        [0.9000, 0.2000],
        [0.1000, 0.8000]], dtype=torch.float64)

In [27]:
model.forward_backward(obs_prob_seq) # runs the forward backward algorithm on observation sequence


In [28]:
posterior = model.forward * model.backward


In [29]:
# marginal per timestep
marginal = torch.sum(posterior, 1)
marginal

tensor([2.2222, 1.7893, 2.3405, 1.6626, 1.5378, 2.9350], dtype=torch.float64)

In [30]:
# Normalize porsterior into probabilities
posterior = posterior / marginal.view(-1, 1) 
posterior

tensor([[0.0703, 0.9297],
        [0.1115, 0.8885],
        [0.7970, 0.2030],
        [0.8936, 0.1064],
        [0.8100, 0.1900],
        [0.1926, 0.8074]], dtype=torch.float64)

In [31]:
results = [model.forward.cpu().numpy(), model.backward.cpu().numpy(), posterior.cpu().numpy()]
result_list = ["Forward", "Backward", "Posterior"]
results

[array([[0.11111111, 0.88888889],
        [0.06163022, 0.93836978],
        [0.68386767, 0.31613233],
        [0.85819949, 0.14180051],
        [0.89028987, 0.10971013],
        [0.19256815, 0.80743185]]),
 array([[1.40699516, 2.32412561],
        [3.23616294, 1.69423506],
        [2.72775586, 1.50282191],
        [1.73108003, 1.24784833],
        [1.39911871, 2.66283883],
        [2.93497129, 2.93497129]]),
 array([[0.07034976, 0.92965024],
        [0.11146783, 0.88853217],
        [0.79701447, 0.20298553],
        [0.89357028, 0.10642972],
        [0.81002232, 0.18997768],
        [0.19256815, 0.80743185]])]

In [32]:
for state_prob, path in zip(results, result_list) :
    inferred_states = np.argmax(state_prob, axis=1) # Returns the indices of the maximum values along axis 1
    print()
    print(path)
    dptable(state_prob)
    print()

print("="*60)
print("Most likely Final State: ",states[inferred_states[-1]])
print("="*60)


Forward
       0        1        2        3        4        5
rain: 0.11111 0.06163 0.68386 0.85819 0.89029 0.19256
no_rain: 0.88888 0.93837 0.31613 0.14180 0.10971 0.80743


Backward
       0        1        2        3        4        5
rain: 1.40699 3.23616 2.72775 1.73108 1.39911 2.93497
no_rain: 2.32412 1.69423 1.50282 1.24784 2.66283 2.93497


Posterior
       0        1        2        3        4        5
rain: 0.07035 0.11146 0.79701 0.89357 0.81002 0.19256
no_rain: 0.92965 0.88853 0.20298 0.10643 0.18997 0.80743

Most likely Final State:  no_rain
